In [17]:
# Import the dependencies
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np

In [18]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90, 90, size=1500)
lngs = np.random.uniform(-90, 90, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [19]:
# Add the latitudes to a list
coordinates = list(lat_lngs)

In [20]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Import the datetime module from the datetime library.
from datetime import datetime

In [21]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [22]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

625

In [12]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | sambava
Processing Record 2 of Set 1 | bellinzona
Processing Record 3 of Set 1 | santa cruz
Processing Record 4 of Set 1 | san carlos
Processing Record 5 of Set 1 | marcona
City not found. Skipping...
Processing Record 6 of Set 1 | madimba
Processing Record 7 of Set 1 | orbetello
Processing Record 8 of Set 1 | chapais
Processing Record 9 of Set 1 | maldonado
Processing Record 10 of Set 1 | lebu
Processing Record 11 of Set 1 | camapua
Processing Record 12 of Set 1 | saint-philippe
Processing Record 13 of Set 1 | berlevag
Processing Record 14 of Set 1 | bocana de paiwas
Processing Record 15 of Set 1 | victoria
Processing Record 16 of Set 1 | skjervoy
Processing Record 17 of Set 1 | zelenogradsk
Processing Record 18 of Set 1 | ushuaia
Processing Record 19 of Set 1 | jamestown
Processing Record 20 of Set 1 | tabou
Processing Record 21 of Set 1 | musoma
Processing Record 22 of Set 1 | sao joao da barr

In [47]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df.reindex(columns=new_column_order)
city_data_df.head(10)



,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Sambava,MG,2021-09-20 21:31:07,-14.2667,50.1667,69.51,79,65,10.27
1,Bellinzona,CH,2021-09-20 21:31:07,46.1928,9.0170,60.71,100,40,1.14
2,Santa Cruz,BO,2021-09-20 21:31:07,-17.8000,-63.1667,90.86,48,20,13.80
3,San Carlos,VE,2021-09-20 21:31:08,9.6612,-68.5827,78.93,88,97,2.28
4,Madimba,TZ,2021-09-20 21:31:08,-10.4000,40.3333,69.12,88,20,9.22
5,Orbetello,IT,2021-09-20 21:31:09,42.4408,11.2196,68.02,70,1,6.13
6,Chapais,CA,2021-09-20 21:24:35,49.7834,-74.8492,74.98,34,7,11.72
7,Maldonado,UY,2021-09-20 21:31:09,-34.6667,-54.9167,56.59,91,99,16.35
8,Lebu,CL,2021-09-20 21:24:09,-37.6167,-73.6500,52.05,76,0,21.79
9,Camapua,BR,2021-09-20 21:31:11,-19.5314,-54.0439,91.02,12,4,3.31


In [48]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")